# Import libraries

In [1]:
import pandas as pd
from torch.utils.data import DataLoader
from dataset import NCFDataset
from recom_ncf import NCFRecommender
from evaluation import Evaluation
from helpers.index_manager import IndexManager
from helpers.splitter import Splitter
import ast
from helpers.dataloader_custom_functions import collate_fn

# Loading data

In [2]:
metadata_features = ['mm_total_recommendation', 'publishers']
# metadata_features = ['pub_encoded', 'dev_encoded', 'tag_encoded', 'lang_encoded', 'gen_encoded', 'cat_encoded', 'mm_total_recommendation', 'z_total_recommendation', 'log_total_recommendation', 'mm_price', 'z_price', 'log_price', 'mm_released_date', 'z_released_date', 'log_released_date']

In [3]:
df = pd.read_csv('../data/interaction-metadata.csv')[['user_id', 'item_id', 'rating_imp', 'timestamp'] + metadata_features]
df.head()

,user_id,item_id,rating_imp,timestamp,mm_total_recommendation,publishers
0,76561197960432447,10,1,1738278781,0.036216,[0]
1,76561198071230926,10,1,1736206418,0.036216,[0]
2,76561198206216352,10,1,1738041574,0.036216,[0]
3,76561198110801124,10,1,1738015332,0.036216,[0]
4,76561199813732773,10,1,1737853720,0.036216,[0]


In [4]:
df['publishers'] = df['publishers'].apply(ast.literal_eval)
# df['developers'] = df['developers'].apply(ast.literal_eval)
# df['tags'] = df['tags'].apply(ast.literal_eval)
# df['supported_languages'] = df['supported_languages'].apply(ast.literal_eval)
# df['genres'] = df['genres'].apply(ast.literal_eval)
# df['categories'] = df['categories'].apply(ast.literal_eval)

# Indexing data

In [5]:
index_manager = IndexManager()
index_manager.fit(df_interaction=df)
index_manager.transform_interactions(df, inplace=True)

Indexed 832660 users and 68396 items
User index range: 0-832659
Item index range: 0-68395


,user_id,item_id,rating_imp,timestamp,mm_total_recommendation,publishers
0,0,0,1,1738278781,0.036216,[0]
1,1,0,1,1736206418,0.036216,[0]
2,2,0,1,1738041574,0.036216,[0]
3,3,0,1,1738015332,0.036216,[0]
4,4,0,1,1737853720,0.036216,[0]
...,...,...,...,...,...,...
1149950,15755,24642,1,1663545142,0.000000,[36058]
1149951,150280,24642,1,1689826252,0.000000,[36058]
1149952,1336,24643,1,1595699739,0.000000,[1]
1149953,11520,24643,0,1706470035,0.000000,[1]


# Creating datasets

In [6]:
splitter = Splitter(df)
df_train, df_val, df_test = splitter.leave_k_out_split()

Splitting data with leave-2-out strategy (1 for validation, 1 for testing)
Total users: 832660
Interactions per user: min=1, max=1034, avg=1.4
Note: 703199 users have fewer than 2 interactions.
These users will be placed entirely in the training set.
Split complete: 1149955 total interactions
Train set: 891033 interactions (77.5%)
Validation set: 129461 interactions (11.3%)
Test set: 129461 interactions (11.3%)


In [7]:
feature_dims = {}
for feature in metadata_features:
    sample = df.iloc[0][feature]
    if isinstance(sample, list):
        num_unique_values = df[feature].explode().nunique()
        feature_dims[feature] = (num_unique_values, 8)
    else:
        feature_dims[feature] = (1, 8)

In [8]:
df_features = df[['item_id'] + metadata_features].drop_duplicates(subset='item_id')

In [9]:
train_dataset = NCFDataset(df_train, feature_dims=feature_dims, df_features=df_features)
val_dataset = NCFDataset(df_val, feature_dims=feature_dims, df_features=df_features)

In [10]:
dataloader_params = {
    'batch_size': 2**12,
    'num_workers': 4,
    'persistent_workers': True,
    'prefetch_factor': 4,
    'pin_memory': True,
    'pin_memory_device': 'cuda',
    'collate_fn': collate_fn,
}
train_dataloader = DataLoader(train_dataset, shuffle=True, **dataloader_params)
val_dataloader = DataLoader(val_dataset, shuffle=False, **dataloader_params)

# Training model

In [11]:
params = {'factors': 8, 'mlp_user_item_dim': 128, 'learning_rate': 0.005, 'epochs': 5, 'optimizer': 'adagrad', 'dropout': 0.0, 'weight_decay': 0.0001, 'loss_fn': 'mse'}

unique_users = index_manager.get_indexed_users()
unique_items = index_manager.get_indexed_items()

model = NCFRecommender(unique_users, unique_items, mlp_additional_features=feature_dims, df_features=df_features, **params)
model.fit(train_dataloader, val_dataloader)

Epoch 1/5
Train loss: 0.140038, Validation loss: 0.142984
Epoch 2/5
Train loss: 0.120456, Validation loss: 0.139276
Epoch 3/5
Train loss: 0.063029, Validation loss: 0.165442
Epoch 4/5
Train loss: 0.035093, Validation loss: 0.166115
Epoch 5/5
Train loss: 0.030292, Validation loss: 0.170387
Early stopping triggered after 5 epochs
Training completed!


# Evaluating model

In [12]:
evaluator = Evaluation(recommender=model, test_data=df_test)
metrics = evaluator.evaluate()

Creating ground truth sets...
Generating predictions...
Processing predictions for 129461 users and 68396 items
Processing 1 of 129461 users... (0.00%)
Time taken to process a user batch: 1.9045023918151855 seconds
Memory usage: 0.29150390625 . Increasing batch size with increasing rate of 1.1
Increased user batch size from 128 to 141
Increased item batch size from 1024 to 1126
Processing 129 of 129461 users... (0.10%)
Time taken to process a user batch: 1.8586008548736572 seconds
Memory usage: 0.205322265625 . Increasing batch size with increasing rate of 1.1
Increased user batch size from 141 to 155
Increased item batch size from 1126 to 1239
Processing 270 of 129461 users... (0.21%)
Time taken to process a user batch: 1.7568995952606201 seconds
Memory usage: 0.221435546875 . Increasing batch size with increasing rate of 1.1
Increased user batch size from 155 to 170
Increased item batch size from 1239 to 1363
Processing 425 of 129461 users... (0.33%)
Time taken to process a user batc

In [13]:
metrics

{'Hit Ratio@10': 0.009307822433010714,
 'NDCG@10': 0.00691624159225661,
 'Recall@10': 0.009307822433010714}